In [ ]:
import os
import numpy as np
import pandas as pd
#from datetime import datetime, timedelta
import datetime
import xarray as xr
#from math import atan2, log
import sys
import uuid

import matplotlib.pyplot as plt
import seawater as sw
import cartopy.crs as ccrs                   # import projections
import cartopy.feature as cf                 # import features
#%matplotlib

sys.path.append('./..//subroutines/')
from read_routines import read_all_usv

#read in email
text_file = open('C:/Users/gentemann/Google Drive/f_drive/docs/projects/saildrone/baja/docs/email_for_file.txt', "r")
email_str = text_file.readlines()

dir_data_out = 'C:/Users/gentemann/Google Drive/public/2019_saildrone/felix/' #'f:/data/cruise_data/saildrone/saildrone_data/'
dir_data = 'C:/Users/gentemann/Google Drive/public/2019_saildrone/' #'f:/data/cruise_data/saildrone/saildrone_data/'
dir_data_pattern = 'C:/Users/gentemann/Google Drive/public/2019_saildrone/*.nc' 
adir_usv = dir_data_pattern


In [ ]:
#input filename
dir_in='/home/ananda/jfpiolle/TMP/'
dir_out='/home/ananda/jfpiolle/TMP/'
filename_in='/home/ananda/jfpiolle/TMP/l3r_temp.nc'

#definitions and things you might have to set differently for each file
itow_mask1=45 #see code just a bit below for figure to determine where to set flags here
itow_mask2=-110 #see code just a bit below for figure to determine where to set flags here
ISDP = 'Saildrone'
SST_type = 'SSTdepth'
Annex_version = '01.1'
File_version = '01.0'
#astr_platform='SD1002'
#astr_title = 'Data from Saildrone cruise from SF to Guadalupe Island April-June 2018'
#astr_uuid = str(uuid.uuid4())


In [ ]:
#fname = 'C:/Users/gentemann/Google Drive/public/2019_saildrone/saildrone_west_coast_survey_2019_sd1047.nc'
#ds=xr.open_dataset(fname)
#ds

In [ ]:
data_dict = read_all_usv(dir_data_pattern)

In [ ]:
for iname,name in enumerate(data_dict):
#    if iname>3:
#        continue
    ds = data_dict[name]
    print('******************************')
    print(iname,name)
    indicative_date_time=(ds.time[0].dt.strftime("%Y%m%d%H%M%S")).data
        
    if ds.TEMP_CTD_MEAN.attrs.get('model_name'):
        Product_string = str(ds.TEMP_CTD_MEAN.attrs['model_name']).replace(' ','_').replace('/','_').replace('(','_').replace(')','_')
    else:
        Product_string = 'model_not_avail'       
    if ds.TEMP_CTD_MEAN.attrs.get('serial_number'):
        tem = str(ds.TEMP_CTD_MEAN.attrs['serial_number'])
        ipos = tem.find('\n')
        tem=tem[:ipos]
        Product_string = str(Product_string + '_' + str(tem)).replace(' ','_').replace('/','_').replace('(','_').replace(')','_') 
    else:
        Product_string = Product_string + '_' + 'serial_not_avail'
    
    filename_L3R = dir_data_out + indicative_date_time + \
        '-' + ISDP + '_' + str(ds.attrs['vehicle_id']) + '-' + 'L2R' + '-' + SST_type + '-' +Product_string+ '-v' +Annex_version+ '-fv' +File_version+ '.nc'
    filename_L3R
    print(filename_L3R)
    
    
    #add global attributes that are missing
    #some of these will need to be changed for new cruises
    astr_uuid = str(uuid.uuid4())

    gstr = {'title': name,
            'name_input_file': name+'.nc',
            'name_of_converter_to_L2R':'https://github.com/cgentemann/Saildrone/blob/master/proc_files/Saildrone_netcdf_format_to_L2R_GHRSST_netcdf_format-v3.ipynb',
    'summary':'none',
    'references':'none',
    'institution':'Saildrone',
    'history':'Saildrone 6-hourly v1 files were used to create this file',
    'comment':'none',
    'license':'free and open',
    'id':'SSTdepth',
    'naming_authority':'org.shipborne-radiometer',
    'product_version':'1.0',
    'uuid':astr_uuid,
    'l2r_version_id':'1.1', 
    'netcdf_version_id':'4.6.1',
    'date_created':datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"), #yyyy-mm-ddThh:mm:ssZ
    'file_quality_level':3,
    'spatial_resolution':'1 min',
    'start_time':str((ds.time[0].dt.strftime("%Y-%m-%dT%H:%M:%SZ")).data),
    'time_coverage_start':str((ds.time[0].dt.strftime("%Y-%m-%dT%H:%M:%SZ")).data),
    'stop_time':str((ds.time[-1].dt.strftime("%Y-%m-%dT%H:%M:%SZ")).data),
    'time_coverage_end':str((ds.time[-1].dt.strftime("%Y-%m-%dT%H:%M:%SZ")).data),
    'northernmost_latitude':ds.lat.max().data,
    'geospatial_lat_max':ds.lat.max().data,
    'southernmost_latitude':ds.lat.min().data,
    'geospatial_lat_min':ds.lat.min().data,
    'easternmost_longitude':ds.lon.max().data,
    'geospatial_lon_max':ds.lon.max().data,
    'westernmost_longitude':ds.lon.min().data,
    'geospatial_lon_min':ds.lon.min().data,
    'geospatial_lat_units':'degrees_north',
    'geospatial_lon_units':'degrees_east',
    'source':'SSTdepth, wind_speed',
    'platform':'Saildrone'+ '_' + str(ds.attrs['vehicle_id']),
    'metadata_link':'TBD',
    'keywords':'Oceans > Ocean Temperature > Sea Surface Temperature',
    'keywords_vocabulary':'NASA Global Change Master Directory (GCMD) Science Keywords',
    'acknowledgment':'The Schmidt Family Foundation, Saildrone, NASA Physical Oceanography',
    'project':'International Shipborne Radiometer Network',
    'publisher_name':'The ISRN Project Office',
    'publisher_url':'http://www.shipborne.radiometer.org',
    'publisher_email':email_str,
    'processing_level':'1.0'}
    #'sensor':str(dataset.TEMP_CTD_MEAN.sensor_description + '_' +\
    #            dataset.TEMP_CTD_MEAN.model_number + '_' + dataset.TEMP_CTD_MEAN.serial_number + ', ' + \
    #            dataset.UWND_MEAN.sensor_description + '_' + dataset.UWND_MEAN.model_number + '_' + \
    #                    dataset.UWND_MEAN.serial_number)
    #del 'nodc_template_version']
    ds.attrs=gstr

    # convert names
    COORD_ATTR = 'lat lon time'
    FLOAT_FILLVALUE = np.finfo(np.float32).min

    swapvar = {
        'TEMP_CTD_MEAN': 'sea_water_temperature',
        'COG_MEAN': 'course_over_ground',
        'lat': 'lat',
        'lon': 'lon',
        'HDG_MEAN': 'true_bearing',
        'ROLL_MEAN': 'platform_roll',
        'PITCH_MEAN': 'platform_pitch',
        'SOG_MEAN': 'speed_over_ground',
        'TEMP_CTD_STDDEV':'sst_total_uncertainty'
    }


    ws_comment = 'Instrument located at to of Saildrone mast at 5.2 m' + '.  This was adjusted ' +\
                'to 10 m as ws_10m = ws*np.log(10./1e-4))/np.log(WS_height/1e-4'

    ws_attr={'standard_name':'wind_speed',
             'long_name':'wind_speed',
             'valid_min':0,
             'valid_max':100,
             'units':'m s-1',
             '_FillValue':FLOAT_FILLVALUE,
             'source':'anemometer',
             'comment':ws_comment,
             'height':'10 m',"coordinates":COORD_ATTR}
    wdir_attr={'standard_name':'wind_to_direction',
             'long_name':'local wind direction',
             'valid_min':0,
             'valid_max':360,
             'units':'degrees',
             '_FillValue':FLOAT_FILLVALUE,
             'source':'anemometer',
             'comment':ws_comment,
             'height':'10 m',"coordinates":COORD_ATTR}

    # creates the new variables
    # -------------------------
    # 1. wind speed
    WS=np.sqrt(ds.UWND_MEAN**2+ds.VWND_MEAN**2)
    WS_height=int(5.2)  #5.2 m installed
    WS_10m = (WS*np.log(10./1e-4))/np.log(WS_height/1e-4)
    ds['wind_speed']=WS_10m
    ds.wind_speed.fillna(FLOAT_FILLVALUE)
    ds.wind_speed.attrs = ws_attr

    ds['wind_direction']=np.arctan2(ds.VWND_MEAN, ds.UWND_MEAN)*np.pi/180.
    ds.wind_direction.fillna(FLOAT_FILLVALUE)
    ds.wind_direction.attrs=wdir_attr

    # SWAP VARIABLE NAMES
    # DROP VARIABLES NOT LISTED IN LIST_VAR
    for var in ds:
        var2 = var
#        print(var)
        if swapvar.get(var): 
            ds = ds.rename({var:swapvar.get(var)})
            var2 = swapvar.get(var)
    #    if any(vv==var2 for vv in list_var):
    #        ds #just a place holder does nothing
        else:
            ds = ds.drop(var2)     


    # 4. Quality level
    flag_bytes=np.byte((0,1,2,3,4,5)) #bytearray([0,1,2,3,4,5])
    iqual_byte = np.ones(shape=ds.time.shape, dtype='b')*5  #change byte to b1
    iqual_byte[:itow_mask1] = 2  #set at top of file from looking at data
    iqual_byte[itow_mask2:] = 2
    vattrs = {
        'long_name': 'measurement quality value',
        'flag_meanings': 'no_data bad_data worst_quality low_quality acceptable_quality best_quality',
        'flag_values': flag_bytes,
        '_FillValue': -128
    }
    ds['quality_level'] = xr.DataArray(
        iqual_byte,
        dims=['time'],
        attrs=vattrs
        )

    l3r=ds
    l3r.sea_water_temperature.attrs['valid_min']=260.0
    l3r.sea_water_temperature.attrs['valid_max']=320.0
    l3r.sea_water_temperature.attrs['units']='kelvin'
    l3r.sea_water_temperature.attrs['long_name']='sea surface depth temperature at 0.6m'
    l3r.time.attrs['standard_name']='time'
    l3r.time.attrs['long_name']='time'
    l3r.lon.attrs['standard_name']='longitude'
    l3r.lon.attrs['long_name']='longitude'
    l3r.lat.attrs['long_name']='latitude'
    l3r.lat.attrs['standard_name']='latitude'
    l3r.true_bearing.attrs['long_name']='platform true bearing'
    l3r.true_bearing.attrs['standard_name']='platform_orientation'
    l3r.speed_over_ground.attrs['long_name']='platform speed over ground'
    l3r.sst_total_uncertainty.attrs['standard_name']='sea_water_temperature standard error'
    l3r.sst_total_uncertainty.attrs['long_name']=' sea water temperature total uncertainty'
    l3r.sst_total_uncertainty.attrs['valid_min']=0.0
    l3r.sst_total_uncertainty.attrs['valid_max']=2.0
    l3r.sst_total_uncertainty.attrs['units']='kelvin'

    l3r.to_netcdf(filename_L3R)

# Make some plots of the data

In [ ]:
#get list of all filenames in directory
from glob import glob
files = [x for x in glob(dir_data_out+'*.nc')]
print('number of file:',len(files))   

In [ ]:
fig = plt.figure(figsize=(14,10))
ax = plt.axes(projection = ccrs.PlateCarree())  # create a set of axes with Mercator projection
for ifile,file in enumerate(files):
#    if ifile>2:
#        continue
    print(ifile)
    ds = xr.open_dataset(file)
    ds.close()
    im=ax.scatter(ds.lon,ds.lat,c=ds.sea_water_temperature,s=.15,transform=ccrs.PlateCarree(),label=ds.attrs['platform'],cmap='jet')
ax.coastlines(resolution='10m')                
ax.set_extent([-180,180,-80,80])
#ax.legend()
cax = fig.add_axes([0.91, 0.26, 0.02, 0.48])
cbar = fig.colorbar(im,cax=cax, orientation='vertical')
cbar.set_label('SST ($^\circ$C)')
plt.show()
fig.savefig(dir_data_out+'map_all_data_2020-06-05.png')    



In [ ]:
#file = 'C:/Users/gentemann/Google Drive/public/2019_saildrone/saildrone_tpos_sd1068_2019.nc'
#ds = xr.open_dataset(file)
#ds.close()
#ds